In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import data_process
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import cv2

In [2]:
data_process.data_to_tfrecord(imagepath='../data/images/', outputpath='../data/face_data_fortest.tfrecord')

이미지를 tfrecord에 저장중: 100%|██████████| 109/109 [00:00<00:00, 384.24it/s]

이미지 갯수:  109
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
filenames = '../data/face_data_7.tfrecord'
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
# Create a dictionary describing the features.
image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string),    
    'class': tf.io.FixedLenFeature([], tf.int64),
    'landmark_points': tf.io.FixedLenFeature([], tf.float32),    
}

def _parse_image_function(example_proto): 
  # Parse the input tf.train.Example proto using the dictionary above.  
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_dataset.map(_parse_image_function)
parsed_image_dataset

In [ ]:
for image_features in parsed_image_dataset:
    print(image_features)
    image_raw = image_features['class'].numpy()
print(image_raw)
# display.display(display.Image(data=image_raw))

In [2]:
@tf.function
def landmark_tfrecord_reader(example):
    feature_description = {"image": tf.io.VarLenFeature(dtype=tf.string),
                           "class": tf.io.VarLenFeature(dtype=tf.int64),
                           "landmark_points": tf.io.VarLenFeature(dtype=tf.float32)}

    inputs = {}
    example = tf.io.parse_single_example(example, feature_description)
    image_raw = tf.sparse.to_dense(example["image"])[0]
    inputs['image'] = tf.io.decode_png(image_raw, channels=3)
    image_class = tf.sparse.to_dense(example["class"])
    inputs['image_landmarks'] = tf.cast(tf.sparse.to_dense(example["landmark_points"]), tf.float32)
    return inputs, image_class

In [3]:
def image_scaling(x, y):
    x['image'] = tf.cast(x['image'], tf.float32) / 255.
    # x 480pixel, y 640pixel
    x['image_landmarks'] = tf.cast(x['image_landmarks'], tf.float32) / 640.
    return x, y

In [4]:
filenames = '../data/face_data_fortest.tfrecord'
ds = tf.data.TFRecordDataset(filenames).map(landmark_tfrecord_reader)
ds = ds.map(image_scaling).batch(32)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [5]:
lds = list(ds)

In [7]:
lds[0][0]['image'][0]

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.7764706 , 0.78039217, 0.70980394],
        [0.77254903, 0.7764706 , 0.7058824 ],
        [0.76862746, 0.77254903, 0.7019608 ],
        ...,
        [0.8509804 , 0.8666667 , 0.8117647 ],
        [0.84705883, 0.8627451 , 0.80784315],
        [0.84313726, 0.85882354, 0.8039216 ]],

       [[0.77254903, 0.7764706 , 0.7058824 ],
        [0.77254903, 0.7764706 , 0.7058824 ],
        [0.77254903, 0.7764706 , 0.7058824 ],
        ...,
        [0.84705883, 0.8627451 , 0.80784315],
        [0.84313726, 0.85882354, 0.8039216 ],
        [0.8392157 , 0.85490197, 0.8       ]],

       [[0.7764706 , 0.78039217, 0.70980394],
        [0.7764706 , 0.78039217, 0.70980394],
        [0.7764706 , 0.78039217, 0.70980394],
        ...,
        [0.84313726, 0.85882354, 0.8039216 ],
        [0.8392157 , 0.85490197, 0.8       ],
        [0.8352941 , 0.8509804 , 0.79607844]],

       ...,

       [[0.07450981, 0.09411765, 0.07843138],
        [0.05